In [ ]:
import keras 
import numpy as np
import pandas as pd
import glob
from music21 import converter, instrument, note, chord,stream
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation,Dense,LSTM,Dropout,Softmax,Flatten

In [ ]:
songs=glob.glob('./trainset-jazz/*.mid')

In [ ]:
song=glob.glob('./test_output4.mid')

In [ ]:
def GetNotes():
    notes=[]
    for file in song:
        midi=converter.parse(song[0])
        notes_to_parse=[]
        try:
            parts=instrument.partitionByInstrument(midi)
        except:
            pass
        if parts:
            notes_to_parse=parts.parts[0].recurse()
        else:
            notes_to_parse=midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                notes.append('.',join(str(n)for n in element.normalOrder))
    #with open('data','wb') as filepath:
     #   pickle.append(notes,filepath)
    return notes
            
            
        
        

In [ ]:
train=GetNotes()
len(train)

In [ ]:
n_values=sorted(set(item for item in train))
n_vocab=len(n_values)

In [ ]:
notes_int=dict((note,number) for number,note in enumerate(n_values))

In [ ]:
seq_length=64
input_network=[]
out_network=[]

In [ ]:
for i in range(0,len(train)-seq_length,1):
    se_in=train[i: i+seq_length]
    se_out=train[i+seq_length]
    input_network.append([notes_int[char] for char in se_in])
    out_network.append([notes_int[se_out]])

In [ ]:
input_network=np.reshape(input_network,(len(input_network),seq_length,1))
input_network=input_network/float(n_vocab)
out_network=keras.utils.np_utils.to_categorical(out_network)

In [ ]:
def model(in_net,len_seq):
    model=Sequential()
    model.add(LSTM(seq_length,input_shape=in_net.shape[1:],return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(seq_length,return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(seq_length,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(len_seq))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam')
    
    return model

In [ ]:
model=model(input_network,n_vocab)

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = 'weights.best.music3.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
model.fit(input_network,out_network,epochs=200,batch_size=128)

In [ ]:
model.load_weights('weights.best.music3.hdf5')

start=np.random.randint(0,len(input_network)-1)
int_to_note = dict((number, note) for number, note in enumerate(n_values))

pattern=input_network[start]
prediction_out=[]
for note_index in range(100):
    start=np.random.randint(0,len(input_network)-1)
    pattern=input_network[start]
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))  
    #prediction_input = prediction_input / float(n_vocab)  
    prediction = model.predict(prediction_input, verbose=0)  
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_out.append(result)  
    np.append(pattern,index)
    #n=len(pattern)
    #pattern = pattern[1:n]
prediction

In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output4.mid')

In [ ]:
create_midi(prediction_out)
